# Bud Flow Lang vs NumPy - Performance Comparison

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/anthropics/bud_flow_lang/blob/main/docs/notebooks/02_numpy_comparison.ipynb)

This notebook provides a comprehensive performance comparison between Bud Flow Lang and NumPy.

## Contents
1. Setup and Installation
2. Methodology
3. Basic Operations Benchmarks
4. Scaling Analysis
5. Memory Bandwidth Analysis
6. Fused Operations
7. Real-World Patterns

## 1. Setup

In [ ]:
# Install (skip if already installed)
!if [ ! -d "bud_flow_lang" ]; then \
    apt-get update -qq && apt-get install -qq -y cmake g++ > /dev/null && \
    git clone --depth 1 https://github.com/anthropics/bud_flow_lang.git && \
    cd bud_flow_lang && mkdir -p build && cd build && \
    cmake .. -DCMAKE_BUILD_TYPE=Release > /dev/null && \
    make -j4 2>&1 | tail -3; \
fi

In [ ]:
import sys
sys.path.insert(0, '/content/bud_flow_lang/build')

import bud_flow_lang_py as flow
import numpy as np
import time
import gc
import matplotlib.pyplot as plt

flow.initialize()

# Print hardware info
info = flow.get_hardware_info()
print(f"Hardware: {info['arch_family']} with {info['simd_width']*8}-bit SIMD")
print(f"Cores: {info['physical_cores']} physical")
print(f"NumPy version: {np.__version__}")

## 2. Benchmarking Methodology

In [ ]:
def benchmark(func, iterations=50, warmup=20):
    """Benchmark a function with proper warmup."""
    # Warmup phase
    for _ in range(warmup):
        func()
    
    # Force garbage collection
    gc.collect()
    
    # Timing phase
    times = []
    for _ in range(iterations):
        start = time.perf_counter()
        func()
        times.append(time.perf_counter() - start)
    
    return {
        'mean': np.mean(times) * 1000,  # ms
        'std': np.std(times) * 1000,
        'min': np.min(times) * 1000,
        'max': np.max(times) * 1000
    }

def compare(name, flow_func, np_func, iterations=50):
    """Compare Flow vs NumPy."""
    flow_stats = benchmark(flow_func, iterations)
    np_stats = benchmark(np_func, iterations)
    
    speedup = np_stats['mean'] / flow_stats['mean']
    
    print(f"{name}")
    print(f"  Flow:  {flow_stats['mean']:.4f} ± {flow_stats['std']:.4f} ms")
    print(f"  NumPy: {np_stats['mean']:.4f} ± {np_stats['std']:.4f} ms")
    print(f"  Ratio: {speedup:.2f}x {'(Flow faster)' if speedup > 1 else '(NumPy faster)'}")
    print()
    
    return flow_stats['mean'], np_stats['mean'], speedup

## 3. Basic Operations Benchmarks

In [ ]:
# Test with 1 million elements
n = 1_000_000

np_a = np.ones(n, dtype=np.float32)
np_b = np.ones(n, dtype=np.float32)
np_c = np.ones(n, dtype=np.float32)

flow_a = flow.ones(n)
flow_b = flow.ones(n)
flow_c = flow.ones(n)

print(f"Array size: {n:,} elements ({n*4/1e6:.1f} MB)")
print("=" * 60)

results = {}

# Sum reduction
f, p, s = compare("Sum Reduction",
                  lambda: flow_a.sum(),
                  lambda: np_a.sum())
results['sum'] = (f, p, s)

# Dot product
f, p, s = compare("Dot Product",
                  lambda: flow.dot(flow_a, flow_b),
                  lambda: np.dot(np_a, np_b))
results['dot'] = (f, p, s)

# Element-wise addition
f, p, s = compare("Element-wise Add",
                  lambda: flow_a + flow_b,
                  lambda: np_a + np_b)
results['add'] = (f, p, s)

# Element-wise multiply
f, p, s = compare("Element-wise Multiply",
                  lambda: flow_a * flow_b,
                  lambda: np_a * np_b)
results['mul'] = (f, p, s)

# FMA
f, p, s = compare("FMA (a*b+c)",
                  lambda: flow.fma(flow_a, flow_b, flow_c),
                  lambda: np_a * np_b + np_c)
results['fma'] = (f, p, s)

In [ ]:
# Visualize results
ops = list(results.keys())
speedups = [results[op][2] for op in ops]

plt.figure(figsize=(10, 5))
colors = ['green' if s > 1 else 'red' for s in speedups]
bars = plt.bar(ops, speedups, color=colors)
plt.axhline(y=1, color='black', linestyle='--', label='Equal performance')
plt.ylabel('Speedup (>1 = Flow faster)')
plt.title('Bud Flow Lang vs NumPy (1M elements)')
plt.legend()

# Add value labels
for bar, val in zip(bars, speedups):
    plt.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.05,
             f'{val:.2f}x', ha='center', va='bottom')

plt.tight_layout()
plt.show()

## 4. Scaling Analysis

How does performance change with array size?

In [ ]:
sizes = [1000, 10000, 100000, 1000000, 10000000]

scaling_results = {
    'dot': {'flow': [], 'numpy': []},
    'add': {'flow': [], 'numpy': []},
    'fma': {'flow': [], 'numpy': []}
}

print("Scaling Analysis")
print("=" * 70)

for n in sizes:
    print(f"\nSize: {n:,}")
    
    np_a = np.ones(n, dtype=np.float32)
    np_b = np.ones(n, dtype=np.float32)
    np_c = np.ones(n, dtype=np.float32)
    flow_a = flow.ones(n)
    flow_b = flow.ones(n)
    flow_c = flow.ones(n)
    
    # Dot product
    flow_t = benchmark(lambda: flow.dot(flow_a, flow_b), iterations=30)['mean']
    np_t = benchmark(lambda: np.dot(np_a, np_b), iterations=30)['mean']
    scaling_results['dot']['flow'].append(flow_t)
    scaling_results['dot']['numpy'].append(np_t)
    print(f"  Dot: Flow={flow_t:.4f}ms NumPy={np_t:.4f}ms Ratio={np_t/flow_t:.2f}x")
    
    # Add
    flow_t = benchmark(lambda: flow_a + flow_b, iterations=30)['mean']
    np_t = benchmark(lambda: np_a + np_b, iterations=30)['mean']
    scaling_results['add']['flow'].append(flow_t)
    scaling_results['add']['numpy'].append(np_t)
    print(f"  Add: Flow={flow_t:.4f}ms NumPy={np_t:.4f}ms Ratio={np_t/flow_t:.2f}x")
    
    # FMA
    flow_t = benchmark(lambda: flow.fma(flow_a, flow_b, flow_c), iterations=30)['mean']
    np_t = benchmark(lambda: np_a * np_b + np_c, iterations=30)['mean']
    scaling_results['fma']['flow'].append(flow_t)
    scaling_results['fma']['numpy'].append(np_t)
    print(f"  FMA: Flow={flow_t:.4f}ms NumPy={np_t:.4f}ms Ratio={np_t/flow_t:.2f}x")
    
    del np_a, np_b, np_c, flow_a, flow_b, flow_c
    gc.collect()

In [ ]:
# Plot scaling
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

for ax, (op, data) in zip(axes, scaling_results.items()):
    ax.loglog(sizes, data['flow'], 'o-', label='Flow', color='blue')
    ax.loglog(sizes, data['numpy'], 's-', label='NumPy', color='orange')
    ax.set_xlabel('Array Size')
    ax.set_ylabel('Time (ms)')
    ax.set_title(f'{op.upper()} Scaling')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 5. Memory Bandwidth Analysis

Understanding memory-bound performance:

In [ ]:
# Compute effective memory bandwidth
print("Memory Bandwidth Analysis")
print("=" * 60)

sizes = [100000, 1000000, 10000000, 50000000]

for n in sizes:
    np_a = np.ones(n, dtype=np.float32)
    np_b = np.ones(n, dtype=np.float32)
    flow_a = flow.ones(n)
    flow_b = flow.ones(n)
    
    # Dot product: reads 2 arrays
    bytes_read = 2 * n * 4  # 2 arrays * n elements * 4 bytes
    
    flow_t = benchmark(lambda: flow.dot(flow_a, flow_b), iterations=20)['mean']
    np_t = benchmark(lambda: np.dot(np_a, np_b), iterations=20)['mean']
    
    flow_bw = bytes_read / (flow_t / 1000) / 1e9  # GB/s
    np_bw = bytes_read / (np_t / 1000) / 1e9
    
    print(f"\nSize: {n:,} elements ({bytes_read/1e6:.1f} MB read)")
    print(f"  Flow:  {flow_t:.3f}ms -> {flow_bw:.1f} GB/s")
    print(f"  NumPy: {np_t:.3f}ms -> {np_bw:.1f} GB/s")
    
    del np_a, np_b, flow_a, flow_b
    gc.collect()

## 6. Fused Operations

Flow's FMA advantage:

In [ ]:
n = 1_000_000

np_a = np.random.randn(n).astype(np.float32)
np_b = np.random.randn(n).astype(np.float32)
np_c = np.random.randn(n).astype(np.float32)

flow_a = flow.flow(np_a)
flow_b = flow.flow(np_b)
flow_c = flow.flow(np_c)

print("Fused Operation Comparison (1M elements)")
print("=" * 60)

# FMA
compare("FMA: a*b + c",
        lambda: flow.fma(flow_a, flow_b, flow_c),
        lambda: np_a * np_b + np_c)

# Separate operations (for comparison)
print("\nBreaking down the operations:")

flow_mul_t = benchmark(lambda: flow_a * flow_b)['mean']
np_mul_t = benchmark(lambda: np_a * np_b)['mean']
print(f"  Multiply only:  Flow={flow_mul_t:.4f}ms NumPy={np_mul_t:.4f}ms")

temp_flow = flow_a * flow_b
temp_np = np_a * np_b

flow_add_t = benchmark(lambda: temp_flow + flow_c)['mean']
np_add_t = benchmark(lambda: temp_np + np_c)['mean']
print(f"  Add only:       Flow={flow_add_t:.4f}ms NumPy={np_add_t:.4f}ms")

print(f"  Separate total: Flow={flow_mul_t + flow_add_t:.4f}ms NumPy={np_mul_t + np_add_t:.4f}ms")

fma_flow = benchmark(lambda: flow.fma(flow_a, flow_b, flow_c))['mean']
print(f"  FMA (fused):    Flow={fma_flow:.4f}ms")
print(f"  Fusion speedup: {(flow_mul_t + flow_add_t) / fma_flow:.2f}x")

## 7. Real-World Patterns

In [ ]:
n = 100_000

print("Real-World Pattern Benchmarks")
print("=" * 60)

# Pattern 1: Vector Normalization (L2)
np_v = np.random.randn(n).astype(np.float32)
flow_v = flow.flow(np_v)

def normalize_np(v):
    return v / np.sqrt(np.dot(v, v))

def normalize_flow(v):
    norm = flow.dot(v, v) ** 0.5
    return v * (1.0 / norm)

compare("L2 Normalization",
        lambda: normalize_flow(flow_v),
        lambda: normalize_np(np_v))

# Pattern 2: Polynomial Evaluation (Horner's method)
np_x = np.random.randn(n).astype(np.float32)
flow_x = flow.flow(np_x)

# Evaluate: 3x^3 + 2x^2 + x + 1 = ((3x + 2)x + 1)x + 1
def poly_np(x):
    return ((3*x + 2)*x + 1)*x + 1

def poly_flow(x):
    # Using FMA for efficiency
    result = flow.fma(x, flow.full(n, 3.0), flow.full(n, 2.0))  # 3x + 2
    result = flow.fma(result, x, flow.ones(n))  # (3x+2)x + 1
    result = flow.fma(result, x, flow.ones(n))  # result*x + 1
    return result

compare("Polynomial (3x³+2x²+x+1)",
        lambda: poly_flow(flow_x),
        lambda: poly_np(np_x))

# Pattern 3: Difference of squares: (a+b)(a-b)
np_a = np.random.randn(n).astype(np.float32)
np_b = np.random.randn(n).astype(np.float32)
flow_a = flow.flow(np_a)
flow_b = flow.flow(np_b)

compare("Difference of Squares (a+b)(a-b)",
        lambda: (flow_a + flow_b) * (flow_a - flow_b),
        lambda: (np_a + np_b) * (np_a - np_b))

## Summary

| Scenario | Winner | Notes |
|----------|--------|-------|
| FMA Operations | **Flow** | True hardware FMA |
| Element-wise (Large) | Tie | Both memory-bound |
| Reductions | NumPy | Multi-threaded BLAS |
| Dot Product | NumPy | BLAS optimization |
| Chained Operations | Flow | Better when fusible |

### Key Insights

1. **Flow excels at fused operations** - Hardware FMA gives 1.3-1.5x speedup
2. **NumPy wins on simple reductions** - Multi-threaded BLAS is hard to beat
3. **Large arrays are memory-bound** - Both libraries approach hardware limits
4. **Consider the use case** - Flow for custom fused kernels, NumPy for standard ops

In [ ]:
flow.shutdown()
print("Benchmark complete!")